In [ ]:
import pandas as pd
import seaborn as sns
import warnings
import category_encoders as ce
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [8]:
import pandas as pd
train = pd.read_csv('train_features.csv')
test = pd.read_csv('test_features.csv')
trainlab = pd.read_csv('train_labels.csv')

In [10]:
train.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')

In [11]:
trainlab.columns

Index(['id', 'status_group'], dtype='object')

In [13]:
train.head(1)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [19]:
train.quantity.value_counts(normalize = True)

enough          0.558687
insufficient    0.254697
dry             0.105152
seasonal        0.068182
unknown         0.013283
Name: quantity, dtype: float64

In [20]:
train['Result'] = train['quantity'] == 'enough'

In [21]:
train.Result.value_counts(normalize = True)

True     0.558687
False    0.441313
Name: Result, dtype: float64

In [ ]:
X = train.drop(columns = ['quantity', 'Result'])
y = train['quantity']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Make pipeline
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    DecisionTreeClassifier(solver='lbfgs', max_iter=10)
)

# Cross-validate with training data
scores = roc_auc_score(pipeline, X_train, y_train, scoring='roc_auc', cv=3, 
                         n_jobs=-1, verbose=10)

In [ ]:
X_train_numeric = X_train.select_dtypes('number')
X_val_numeric   = X_val.select_dtypes('number')

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
tree = DecisionTreeClassifier(max_depth=5)
tree.fit(X_train_numeric, y_train)
y_pred_proba = tree.predict_proba(X_val_numeric)[:,1]
roc_auc_score(y_val, y_pred_proba)

In [ ]:
import graphviz
from sklearn.tree import export_graphviz

dot_data = export_graphviz(tree, out_file=None, feature_names=X_train_numeric.columns, class_names=['n', 'y'], filled=True, impurity=False, proportion=True)

graphviz.Source(dot_data)